In [ ]:
from json import load
from tflearn.layers.core import input_data

with open("VGG.json", "r") as inputFile:
    layers = load(inputFile)
networkRoot = layers['VGG']['VGG11']

In [ ]:
import tflearn
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.estimator import regression

In [ ]:
import numpy as np
from scipy.misc import imresize
import tflearn.datasets.oxflower17 as oxflower17

# Bug at resize_pics don't work
X_, Y = oxflower17.load_data(dirname='Data/17flowers', one_hot=True,
                            resize_pics=(224, 224))

X = np.empty((X_.shape[0], 224, 224, 3), dtype=np.float32)
for i, image in enumerate(X_):
    # return is RGB 224x224 and float64 [0-255]
    temp = imresize(image, (224, 224), mode='RGB').astype(np.float32)
    X[i] = temp / np.max(temp)
X_ = None

In [ ]:
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [ ]:
from Utility import createArchitecture
# VGGNet Config A (11 weight layers)
network = input_data(shape=[None, 224, 224, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
network = createArchitecture(networkRoot, network)

In [ ]:
network = fully_connected(network, 17, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0001)

![VGGNet](Data/VGGNetConfig.png)
![VGG](Data/VGG_Tensorboard.png)

In [ ]:
import os, shutil
if os.path.exists('/tmp/tflearn_logs/vgg11_oxflowers17/'):
    shutil.rmtree('/tmp/tflearn_logs/vgg11_oxflowers17/')

# Training
model = tflearn.DNN(network, checkpoint_path='Data/checkpoint/model_vgg11', max_checkpoints=1,
                    tensorboard_verbose=2)
model.fit(X, Y, n_epoch=500, shuffle=True,
          show_metric=True, batch_size=1, snapshot_step=500,
          snapshot_epoch=False, run_id='vgg11_oxflowers17')